Shell Averages
==========

**Summary:**   Spherical averages of requested output variables.  Each output variable is stored as a 1-D function of radius, $q_i(r,t)$, such that

$q_i(r,t) = \frac{1}{4\pi}\int_0^{2\pi}\int_0^\pi f_i(r,\theta,\phi,t) \mathrm{sin}\theta d\theta d\phi $

for each desired output $f_i$.

**Subdirectory:**  Shell_Avgs

**main_input prefix:** shellavg

**Python Class:** Shell_Avgs

**Additional Namelist Variables:**  
None

The Shell-Averaged outputs are useful for examining how quantities vary as a function of radius.  They are particularly useful for examining the distribution of energy as a function of radius, or the heat flux balance established by the system.

Examining the *main_input* file, we see that the following output values have been denoted for the Shell Averages for the Boussinesq case:


| Menu Code  | Description |
|------------|-------------|
| 1          | Radial Velocity |
| 2          | Theta Velocity |
| 3          | Phi Velocity  |
| 501        | Temperature Perturbation |
| 1440       | Radial Convective Heat Flux|
| 1470       | Radial Conductive Heat Flux |


For the anelastic example, we have a few additional variables of note:


| Menu Code  | Description |
|------------|-------------|
| 1433       | Volumetric Heat Flux |
| 1455       | Radial Enthalphy Flux |
| 1923       | Radial Advective Kinetic Energy Flux |
| 1935       | Radial Viscous Kinetic Energy Flux |

In the example that follows, we will plot the spherically-averaged velocity field as a function of radius, the mean temperature profile, and the radial heat flux.  We begin with a preamble similar to that used for the Global Averages.  Using the help function, we see that the Shell_Avgs data structure is similar to that of the G_Avgs.  There are three important differences:
*  There is a radius attribute (necessary if we want to plot anything vs. radius)
*  The dimensionality of the values array has changed;  radial index forms the first dimension.
*  The second dimension of the values array has a length of 4.  In addition to the spherical mean, the 1st, 2nd and 3rd spherical moments (variance, skewness, kurtosis) are stored in indices 1,2, and 3 respectively.

In [ ]:
from rayleigh_diagnostics import Shell_Avgs, build_file_list
import matplotlib.pyplot as plt
import numpy


model_type = 1 # 1 for anelastic example, 2 Boussinesq example
base_dir = '/home/nfeatherstone/runs/cig_vis/'
font_size=14     # Font size for plot labels 

if (model_type == 1):
    model_dir = base_dir+'anelastic/'
    # Define some units for plotting purposes
    eunits = r'(erg cm$^{-3}$)'                # energy density
    tunits = '(s)'                             # time
    vunits = r'(cm s$^{-1}$)'                  # velocity
    dunits = '(cm)'                            # distance
    thermal_label = r' Specific Entropy '      # specific entropy
    thermal_units = r'(erg g$^{-1}$ K$^{-1}$)' # specific entropy
    lunits = r'(erg s$^{-1}$)'                 # energy / time (luminosity)
    funits = '(nHz)'                           # Frequency units
    mfunits = '(g cm$^{-2}$ s$^{-1}$)'
    
    # Next, we set some timestep ranges for plotting purposes
    imin = 0             # minimum iteration number to process for time series
    imax = 10000000      # maximum iteration number to process for time series
    
if (model_type == 2):
    model_dir = base_dir+'Boussinesq/'
    # Define some units for plotting purposes
    eunits = ''                          # energy density
    tunits = '(viscous diffusion times)' # time
    vunits = ''                          # velocity
    dunits = ''                          # distance
    thermal_label = ' Temperature ' 
    thermal_units = '' 
    lunits = ''                          # energy / time (luminosity)
    funits = ''                          # Frequency units
    mfunits = ''
    
    # Next, we set some timestep ranges for plotting purposes
    imin = 0             # minimum iteration number to process for time series
    imax = 10000000      # maximum iteration number to process for time series


files = build_file_list(imin,imax,path=model_dir+'Shell_Avgs')
sa = Shell_Avgs(filename=files[0], path='')
help(sa)


***

While it can be useful to look at instaneous snapshots of Shell Averages, it's often useful to examine these outputs in a time-averaged sense.    Similarly to the G_Avgs class, the Shell_Avgs class will accept a list of files.  The default behavior is to time-average this data, but it can also be concatenated if desired by setting the option parameter time_average=False when initializing the Shell_Avgs object.

In [ ]:
sa = Shell_Avgs(files,path='')
savg =sa.vals
niter = sa.niter
radius = sa.radius
lut = sa.lut

In [ ]:
#Define the quantity codes of interest

vr      = lut[1]        # Radial Velocity
vtheta  = lut[2]    # Theta Velocity
vphi    = lut[3]      # Phi Velocity
thermal = lut[501] # Temperature


eflux = lut[1440]  # Convective Heat Flux (radial)
cflux = lut[1470]  # Conductive Heat Flux (radial)

# Anelastic additions
if (model_type == 1):
    eflux = lut[1455]  # Enthalpy Flux (radial)
    rflux = lut[1433]  # Effective thermal energy flux due to internal volumetric heating
    keflux = lut[1923] # Kinetic energy flux via advection (radial)
    vflux = lut[1935]  # Kinetic energy flux via viscous spread (radial)

    # Finally, for models with a volumetric heating, such as models of stars, output
    # an associated flux for plotting purposes
    rflux = lut[1433]  # Effective thermal energy flux due to internal volumetric heating

Velocity vs. Radius
---------------------
Next, we plot the mean velocity field, and its first moment, as a function of radius.   Notice that the radial and theta velocity components have a zero spherical mean.  Since we are running Boussinesq or anelastic models, this is a good sign!

In [ ]:
sizetuple = (10,8)
fig, ax = plt.subplots(nrows=2, ncols =1, figsize=sizetuple)
plt.rcParams.update({'font.size': font_size})

ax[0].plot(radius,savg[:,0,vr],label=r'$v_r$')
ax[0].plot(radius,savg[:,0,vtheta], label=r'$v_\theta$')
ax[0].plot(radius,savg[:,0,vphi], label=r'$v_\phi$')
ax[0].legend(shadow=True,loc='center right')
ax[0].set_xlabel('Radius '+dunits)
ax[0].set_ylabel('Velocity '+vunits)
ax[0].set_title('Velocity Components: Spherical Average')

ax[1].plot(radius,savg[:,1,vr]**0.5,label=r'$v_r$')
ax[1].plot(radius,savg[:,1,vtheta]**0.5, label=r'$v_\theta$')
ax[1].plot(radius,savg[:,1,vphi]**0.5, label=r'$v_\phi$')
ax[1].legend(shadow=True,loc='upper left')
ax[1].set_xlabel('Radius '+dunits)
ax[1].set_ylabel('Velocity '+vunits)
ax[1].set_title('Velocity Components: Standard Deviation from Spherical Mean')


plt.tight_layout()
plt.show()

Radial Thermal Profile
------------------------------
We might also look at variation of temperature or entropy (for anelastic models) in radius ...

In [ ]:
sizetuple=(10,4)
fig, ax = plt.subplots(figsize=sizetuple)

ax.plot(radius,savg[:,0,thermal,0],label='Spherical Mean')
ax.plot(radius,savg[:,1,thermal,0]**0.5, label='Standard Deviation')
ax.legend(shadow=True,loc='center left')
ax.set_xlabel('Radius')
ax.set_ylabel(thermal_label+thermal_units)
ax.set_title('Radial Thermal Profile')


plt.show()

Heat Flux Contributions
--------------------------
We can also examine the balance between convective and conductive heat flux.  In this case, before plotting these quantities as a function of radius, we normalize them by the surface area of the sphere to form a luminosity.

In [ ]:
if (model_type == 2):
    # This is for Boussinesq models
    fpr=4.0*numpy.pi*radius*radius
    elum = savg[:,0,eflux,0]*fpr
    clum = savg[:,0,cflux,0]*fpr
    tlum = elum+clum
    fig, ax = plt.subplots()
    ax.plot(radius,elum,label='Convection')
    ax.plot(radius,clum, label='Conduction')
    ax.plot(radius,tlum, label='Total')
    ax.set_title('Energy Flux Balance')
    ax.set_ylabel('Luminosity '+lunits)
    ax.set_xlabel('Radius')
    ax.legend(shadow=True)
    plt.tight_layout()
    plt.show()
    
if (model_type == 1):
    # This is for anelastic
    fpr=4.0*numpy.pi*radius*radius
    elum = savg[:,0,eflux,0]*fpr
    clum = savg[:,0,cflux,0]*fpr
    klum = savg[:,0,keflux,0]*fpr
    rlum = savg[:,0,rflux,0]*fpr
    vlum = -savg[:,0,vflux,0]*fpr
    tlum = elum+clum+klum+rlum+vlum


    sizetuple=(10,6)
    fig, ax = plt.subplots(figsize=sizetuple)
    ax.plot(radius,elum,label='Enthalpy Flux')
    ax.plot(radius,clum, label='Conductive Flux')
    ax.plot(radius,klum,label='KE Flux')
    ax.plot(radius,vlum, label='Viscous Flux')
    ax.plot(radius,rlum, label='Volumetric Heating')
    ax.plot(radius,tlum, label='Total')
    ax.set_title('Energy Flux Balance')
    ax.set_ylabel('Luminosity '+lunits)
    ax.set_xlabel('Radius '+dunits)
    ax.legend(shadow=True,fontsize=10,loc='center left')
    plt.tight_layout()
    plt.show()